#과제

1. 하단 셀에서 lightGBM 구현하기 (상단 Ensemble 셀 모두 실행 후 진행)
2. 그리드 서치, 랜덤 서치 각각 어떤 상황에서 효율적인지 설명
3. 다른 데이터셋으로 Decision Tree regression task 진행해보기
  - 원하는 데이터셋을 사용해도 괜찮고, 올려둔 bike_data 사용해도 괜찮음
  - 직접 구현해보는 것이 어렵다면 'bike sharing data decision tree regressor' 키워드로 검색 후 참고


## 과제1

In [31]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_validate, KFold

data=pd.read_csv('/content/drive/MyDrive/day.csv')
data["dteday"] = pd.to_datetime(data["dteday"])
data["year"] = data["dteday"].dt.year
data["month"] = data["dteday"].dt.month
data["day"] = data["dteday"].dt.day
data.drop(["instant", "dteday"], axis=1, inplace=True)

y = data["cnt"]
data.drop(["cnt"], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'max_depth': 7,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'verbosity': -1,
}

k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
train_scores = []
test_scores = []

for train_idx, test_idx in kfold.split(data, y):
    d_train = lgb.Dataset(data.iloc[train_idx], label=y.iloc[train_idx])
    d_valid = lgb.Dataset(data.iloc[test_idx], label=y.iloc[test_idx], reference=d_train)

    model = lgb.train(params, d_train, verbose_eval=100, valid_sets=[d_valid], num_boost_round=2000, early_stopping_rounds=10)
    y_train_pred = model.predict(data.iloc[train_idx], num_iteration=model.best_iteration)
    y_test_pred = model.predict(data.iloc[test_idx], num_iteration=model.best_iteration)

    train_rmse = np.sqrt(mean_squared_error(y.iloc[train_idx], y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y.iloc[test_idx], y_test_pred))

    train_scores.append(train_rmse)
    test_scores.append(test_rmse)

print("Train Score: ", np.mean(train_scores))
print("Test Score: ", np.mean(test_scores))


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 10 rounds
[100]	valid_0's rmse: 127.156
[200]	valid_0's rmse: 116.47
[300]	valid_0's rmse: 113.604
Early stopping, best iteration is:
[299]	valid_0's rmse: 113.564
Training until validation scores don't improve for 10 rounds


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's rmse: 170.892
[200]	valid_0's rmse: 160.102
Early stopping, best iteration is:
[266]	valid_0's rmse: 158.769
Training until validation scores don't improve for 10 rounds


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's rmse: 151.496
[200]	valid_0's rmse: 140.482
Early stopping, best iteration is:
[213]	valid_0's rmse: 140.428


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 10 rounds
[100]	valid_0's rmse: 140.038
[200]	valid_0's rmse: 129.152
Early stopping, best iteration is:
[239]	valid_0's rmse: 128.41
Training until validation scores don't improve for 10 rounds
[100]	valid_0's rmse: 152.964
Early stopping, best iteration is:
[151]	valid_0's rmse: 145.345
Train Score:  58.08034262450203
Test Score:  137.3030299365824


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


##과제2
답:

In [ ]:
'''
그리드 서치와 랜덤 서치의 개념은 개인적으로 Batch Gradient Descent와
Stochastic Gradient Descent의 관계를 떠올리게 되었습니다.
하이퍼 파라미터들의 모든 조합을 시험하는 그리드 서치는 Batch Gradient Descent와 마찬가지로
최적의 결과를 확실히 찾는 데에는 랜덤 서치보다 유리할 것입니다. 하지만 시간 등이 오래 소모되므로,
하이퍼 파라미터 수가 적은 경우나 가능 값들의 수가 적은 경우가 더 적합한 상황이라고 생각합니다.
반대로 랜덤 서치는 랜덤으로 선택한 조합만을 보기에, 최적으로 하는 데에는 부족하지만 시간 등의 측면에서는
더 효율적입니다. 따라서 하이퍼 파라미터가 많은 경우나 가능 값들의 수가 커서 그리드 서치를 시행하기
어려운 경우에 하는 것이 적합한 것 같습니다.
올려주신 bike data 같은 경우는 비교적 하이퍼 파라미터가 적기에 grid search가 더 좋은 선택이라고 생각합니다.
'''

## 과제3

In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer


def positive_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

positive_rmse_score = make_scorer(positive_rmse, greater_is_better=False)


X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
regressor = DecisionTreeRegressor(random_state=42)
hyperparameter_grid = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10]
}

grid_search = GridSearchCV(regressor, hyperparameter_grid,  n_jobs=-1)

grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
print("Best Parameters: ", best_parameters)


y_train_pred = grid_search.predict(X_train)
train_rmse = positive_rmse(y_train, y_train_pred)
print("Train RMSE: ", train_rmse)
y_test_pred = grid_search.predict(X_test)
test_rmse = positive_rmse(y_test, y_test_pred)
print("Test RMSE: ", test_rmse)


Best Parameters:  {'max_depth': 15, 'min_samples_leaf': 2, 'min_samples_split': 2}
Train RMSE:  55.45408493997246
Test RMSE:  202.50138806761836
